In [1]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
from copy import *
%load_ext autoreload

%autoreload 2

In [2]:
from dcegm.retirement.ret import util
from dcegm.solve.egm_step import egm_step
from dcegm.solve.solve_auxiliary import create_container
from dcegm.solve.solve import solve_retirement_model

In [3]:
import pickle
with open('../dcegm/test/resources/m0_value_new.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('../dcegm/test/resources/m0_policy_new.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)

with open('../dcegm/test/resources/m5_value_new.pkl', 'rb') as file : 
    m5_value = pickle.load(file)

with open('../dcegm/test/resources/m5_policy_new.pkl', 'rb') as file : 
    m5_policy = pickle.load(file)

# Model parameters (default)

In [4]:
# Number of periods (fist period is t=1) 
num_periods = 25

# Number of grid points over assets
num_grid = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 50

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
cost_work = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [5]:
value, policy = solve_retirement_model(
    num_grid,
    n_quad_points,
    r, coeffs_age_poly,
    theta,
    cost_work,
    beta,
    lambda_,
    sigma,
    mmax,
    num_periods)

In [6]:
for period in range(23, -1, -1):
    for choice in [1,0]:
        np.testing.assert_array_almost_equal(value[period][choice], m0_value[period][choice])
        np.testing.assert_array_almost_equal(policy[period][choice], m0_policy[period][choice])

In [7]:
# Change relevant parameters for m5 specification

# Interest rate
r = 0.05

# Discount factor
beta = 1 / (1 + r)

# Standard deviation of log-normally distributed income shocks
sigma = 0.35

# Scale of the EV taste shocks 
lambda_ = 0.2

In [8]:
value5, policy5 = solve_retirement_model(
    num_grid,
    n_quad_points,
    r, coeffs_age_poly,
    theta,
    cost_work,
    beta,
    lambda_,
    sigma,
    mmax,
    num_periods)

In [9]:
for period in range(23, -1, -1):
    for choice in [1,0]:
        np.testing.assert_array_almost_equal(value5[period][choice], m5_value[period][choice])
        np.testing.assert_array_almost_equal(policy5[period][choice], m5_policy[period][choice])